# train

In [1]:
import yaml
import os

with open("./config.yaml", 'rb') as f:
    config = yaml.load(f)

<ipython-input-1-1499ab5789c7>:5: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config = yaml.load(f)


In [2]:
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
os.makedirs(OUTPUT_ROOT, exist_ok=True)

## load library

In [3]:
# python default library
import os
import shutil
import datetime
import sys
import pickle

# general analysis tool-kit
import numpy as np
import pandas as pd
#from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

# pytorch
import torch
from torch import optim, nn
from torch.utils.tensorboard import SummaryWriter

#from torchsummary import summary

# etc
import yaml
yaml.warnings({'YAMLLoadWarning': False})

# original library
sys.path.append('/home/hiroki/research/dcase2021_task2/src/functions')
import common as com
import pytorch_modeler as modeler
from pytorch_model import Conditional_VAE as Model
#from pytorch_utils import filtered_load_model
#import models

import librosa
import IPython
import librosa.display

## load config and set logger

In [4]:
log_file = config['IO_OPTION']['OUTPUT_ROOT']+'/train_{0}.log'.format(datetime.date.today())
logger = com.setup_logger(log_file, '00_train.py')

## Setting

In [5]:
# Setting seed
modeler.set_seed(777)

In [6]:
############################################################################
# Setting I/O path
############################################################################
# input dirs
INPUT_ROOT = config['IO_OPTION']['INPUT_ROOT']
#dev_dir = INPUT_ROOT + "/dev_data"
#add_dev_dir = INPUT_ROOT + "/add_dev_data"
# machine type
machine_types = ['ToyCar', 'gearbox', 'valve', 'ToyTrain', 'pump', 'fan', 'slider']
# output dirs
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
MODEL_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/models'
TB_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/tb'
OUT_FEATURE_DIR = OUTPUT_ROOT + '/extraction_features'
OUT_SCORE_DIR = OUTPUT_ROOT + '/score'
OUT_PRED_DIR = OUTPUT_ROOT + '/pred'
#os.makedirs(OUTPUT_ROOT, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(TB_DIR, exist_ok=True)
os.makedirs(OUT_FEATURE_DIR, exist_ok=True)
os.makedirs(OUT_SCORE_DIR, exist_ok=True)
os.makedirs(OUT_PRED_DIR, exist_ok=True)
# copy config
shutil.copy('./config.yaml', OUTPUT_ROOT)

'/media/hiroki/HDD1TB/research/dcase2021_task2/output/CenterLossVAE/ex12/config.yaml'

In [7]:
torch.cuda.is_available()

True

In [8]:
ext_data = pd.read_pickle(f'{INPUT_ROOT}/valve_features.pkl')

In [9]:
pd.DataFrame(ext_data['train']['features'])

,0,1,2,3,4,5,6,7,8,9,...,3766,3767,3768,3769,3770,3771,3772,3773,3774,3775
0,0.243716,0.338328,0.272088,0.226127,0.029367,0.339973,0.209255,0.105823,0.137297,0.145012,...,0.022174,0.006773,0.069666,0.301964,0.180498,0.033291,0.098398,0.024299,0.016897,0.123153
1,0.237665,0.327515,0.263212,0.232293,0.027842,0.367579,0.210006,0.094775,0.134482,0.143913,...,0.027657,0.028354,0.165744,0.330388,0.136494,0.035396,0.146613,0.123391,0.007483,0.144597
2,0.225514,0.379070,0.276066,0.196089,0.027455,0.265828,0.224954,0.132161,0.143215,0.143278,...,0.040949,0.034133,0.122343,0.213779,0.166338,0.049096,0.042672,0.044685,0.002102,0.145841
3,0.241087,0.328841,0.287649,0.230211,0.029166,0.339409,0.215851,0.104063,0.135149,0.140004,...,0.047530,0.000834,0.029579,0.157189,0.159237,0.092072,0.176285,0.072260,0.025339,0.222447
4,0.247998,0.328309,0.278905,0.237095,0.027144,0.362545,0.209568,0.100579,0.133083,0.137760,...,0.022089,0.001686,0.082939,0.303310,0.165293,0.078591,0.169059,0.093608,0.015137,0.118250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6013,0.242294,0.320149,0.291148,0.246376,0.024519,0.336719,0.213888,0.098774,0.152700,0.135828,...,0.028749,0.020032,0.089271,0.324535,0.164114,0.011856,0.131130,0.048871,0.016199,0.125525
6014,0.239820,0.328761,0.283221,0.232255,0.026091,0.329097,0.217799,0.107723,0.151887,0.142054,...,0.065597,0.009168,0.159468,0.321171,0.235521,0.020378,0.059581,0.037248,0.034675,0.046954
6015,0.233884,0.312639,0.287863,0.243157,0.024677,0.309460,0.243808,0.103488,0.180332,0.135159,...,0.159631,0.007203,0.071548,0.284185,0.112961,0.240836,0.032650,0.014332,0.023457,0.157205
6016,0.232720,0.344319,0.275893,0.233720,0.027888,0.292808,0.227276,0.124664,0.170116,0.145821,...,0.098789,0.002348,0.054095,0.253814,0.089476,0.159530,0.034791,0.019259,0.031108,0.169736


## training

In [10]:
#############################################################################
# run
#############################################################################
def run(machine_type):
    com.tic()
    
    logger.info('TARGET MACHINE_TYPE: {0}'.format(machine_type))
    logger.info('MAKE DATA_LOADER')
    # dataloader
    ext_data = pd.read_pickle(f'{INPUT_ROOT}/{machine_type}_features.pkl')
    dataloaders_dict = modeler.make_dataloader(ext_data)
    # define writer for tensorbord
    os.makedirs(TB_DIR+'/'+machine_type, exist_ok=True)
    tb_log_dir = TB_DIR + '/' + machine_type
    writer = SummaryWriter(log_dir = tb_log_dir)
    # out path
    model_out_path = MODEL_DIR+'/{}_model.pth'.format(machine_type)
    score_out_path = OUT_SCORE_DIR + '/{}_score.csv'.format(machine_type)
    pred_out_path = OUT_PRED_DIR + '/{}_pred.csv'.format(machine_type)
    logger.info('TRAINING')
    # parameter setting
    in_features = ext_data['train']['features'].shape[1]
    mid_size = config['param']['mid_size']
    latent_size = config['param']['latent_size']
    num_classes = 6
    net = Model(in_features, mid_size, latent_size, num_classes)
    optimizer = optim.Adam(net.parameters(), lr=1e-3)
    num_epochs = config['param']['num_epochs']
    #scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e-2, 
    #                                          max_lr=1e-3, epochs=num_epochs, steps_per_epoch=len(dataloaders_dict['train']))
    # training
    output_dicts = modeler.train_net(net, dataloaders_dict, optimizer, num_epochs, writer, model_out_path, score_out_path, pred_out_path)
    
    com.toc()

In [11]:
machine_types

['ToyCar', 'gearbox', 'valve', 'ToyTrain', 'pump', 'fan', 'slider']

In [12]:
for machine_type in machine_types:
    run(machine_type)

2021-05-26 23:21:17,193 - 00_train.py - INFO - TARGET MACHINE_TYPE: ToyCar
2021-05-26 23:21:17,194 - 00_train.py - INFO - MAKE DATA_LOADER
2021-05-26 23:21:17,239 - 00_train.py - INFO - TRAINING
2021-05-26 23:21:21,473 - pytorch_modeler.py - INFO - train
  0%|          | 0/12 [00:00<?, ?it/s]

use: cuda:0


100%|██████████| 12/12 [00:01<00:00,  9.64it/s]
2021-05-26 23:21:22,722 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 2/2 [00:00<00:00, 49.44it/s]
2021-05-26 23:21:22,783 - pytorch_modeler.py - INFO - valid_target
100%|██████████| 2/2 [00:00<00:00, 67.34it/s]


,AUC,pAUC
Source_0,0.518800,0.512105
Source_1,0.783000,0.595789
Source_2,0.582100,0.577368
Target_0,0.383000,0.508947
Target_1,0.947000,0.863684
Target_2,0.294200,0.473684
mean,0.584683,0.588596
h_mean,0.500477,0.566201


2021-05-26 23:21:23,771 - pytorch_modeler.py - INFO - epoch:1/300, train_losses:3787.386088, val_AUC_hmean:0.500477, val_pAUC_hmean:0.566201, best_flag:True
2021-05-26 23:21:23,772 - pytorch_modeler.py - INFO - train
100%|██████████| 12/12 [00:00<00:00, 16.79it/s]
2021-05-26 23:21:24,488 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 2/2 [00:00<00:00, 49.77it/s]
2021-05-26 23:21:24,549 - pytorch_modeler.py - INFO - valid_target
100%|██████████| 2/2 [00:00<00:00, 59.29it/s]


...

100%|██████████| 12/12 [00:00<00:00, 27.31it/s]
2021-05-26 23:48:37,157 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 2/2 [00:00<00:00, 101.42it/s]
2021-05-26 23:48:37,192 - pytorch_modeler.py - INFO - valid_target
100%|██████████| 2/2 [00:00<00:00, 108.04it/s]
2021-05-26 23:48:37,227 - pytorch_modeler.py - INFO - epoch:267/300, train_losses:10.219178, val_AUC_hmean:0.667154, val_pAUC_hmean:0.594302, best_flag:False
2021-05-26 23:48:37,228 - pytorch_modeler.py - INFO - train
100%|██████████| 12/12 [00:00<00:00, 27.30it/s]
2021-05-26 23:48:37,670 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 2/2 [00:00<00:00, 102.82it/s]
2021-05-26 23:48:37,705 - pytorch_modeler.py - INFO - valid_target
100%|██████████| 2/2 [00:00<00:00, 108.01it/s]
2021-05-26 23:48:37,740 - pytorch_modeler.py - INFO - epoch:268/300, train_losses:10.076747, val_AUC_hmean:0.673354, val_pAUC_hmean:0.583566, best_flag:False
2021-05-26 23:48:37,741 - pytorch_modeler.py - INFO - train
100%|███████

2021-05-26 23:48:45,502 - pytorch_modeler.py - INFO - train
100%|██████████| 12/12 [00:00<00:00, 27.12it/s]
2021-05-26 23:48:45,947 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 2/2 [00:00<00:00, 105.73it/s]
2021-05-26 23:48:45,980 - pytorch_modeler.py - INFO - valid_target
100%|██████████| 2/2 [00:00<00:00, 108.11it/s]
2021-05-26 23:48:46,016 - pytorch_modeler.py - INFO - epoch:284/300, train_losses:7.312886, val_AUC_hmean:0.683607, val_pAUC_hmean:0.587827, best_flag:False
2021-05-26 23:48:46,016 - pytorch_modeler.py - INFO - train
100%|██████████| 12/12 [00:00<00:00, 27.36it/s]
2021-05-26 23:48:46,457 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 2/2 [00:00<00:00, 106.73it/s]
2021-05-26 23:48:46,490 - pytorch_modeler.py - INFO - valid_target
100%|██████████| 2/2 [00:00<00:00, 109.80it/s]
2021-05-26 23:48:46,525 - pytorch_modeler.py - INFO - epoch:285/300, train_losses:7.285376, val_AUC_hmean:0.677998, val_pAUC_hmean:0.593662, best_flag:False
2021-05-26 23:

100%|██████████| 2/2 [00:00<00:00, 108.26it/s]
2021-05-26 23:48:54,289 - pytorch_modeler.py - INFO - epoch:300/300, train_losses:5.719461, val_AUC_hmean:0.684594, val_pAUC_hmean:0.594488, best_flag:False
2021-05-26 23:48:54,290 - pytorch_modeler.py - INFO - best model, epoch:100/300, train_losses:299.324793, val_pAUC_hmean:0.612358,


,AUC,pAUC
Source_0,0.792950,0.650000
Source_1,0.925300,0.788947
Source_2,0.832709,0.816118
Target_0,0.503100,0.493684
Target_1,0.670306,0.573362
Target_2,0.610342,0.500698
mean,0.722451,0.637135
h_mean,0.692785,0.612358


elapsed time: 161.943755865 [sec]


In [13]:
#run(machine_types[0])